# Coursera Capstone - Characteristics of car accidents and their impact on police, ambulance and fire brigades

## Table of contents:

1. Introduction Section
    - Scenario and background
    - Problem to be solved
    - Recipients of the report
2. Data Section
    - Data required to resolve the problem
    - How the data will be used to solve the problem
    - Mapping of the data
3. Methodology Section
4. Result Section
5. Discussion Section
6. Conslusion Section


## 1. Introduction Section

### Scenario and background

### Problem to be solved
The challenge to be solved is to properly prepare and inform the police, fire brigade and ambulance in advance about the possibility of very serious road accidents depending on the prevailing road conditions. Therefore, I want to find accidents with the following assumptions:
- Severity is a minimum of 2
- More than 4 are injured, including pedestrians and cyclists
- Depending on the weather condition and road condition
- Depending on the time of day
- Cases resulting from the use of alcohol and other intoxicants are rejected as not resulting directly from weather factors.

### Recipients of the report
I believe that this is a suitable project for a major city security center in Europe, the United States or Asia, as the approach and methodology used here will help prepare services such as the police, fire brigade or hospitals in advance for emergencies (e.g. severe road accidents) from weather conditions.
